In [2]:
import scipy
import scipy.sparse, scipy.stats
import numpy
import os
import re
import multiprocessing
import subprocess
import pickle
import matplotlib.pyplot
import time
import statistics
import sklearn
import statsmodels , statsmodels.api
import itertools

C:\Users\chenfu\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def moving_integration (values, window):
    weights = numpy.repeat(1.0, window)
    sma = numpy.convolve(values, weights, 'same')
    return sma
def moving_average (values, window):
    weights = numpy.repeat(1.0, window)/window
    sma = numpy.convolve(values, weights, 'same')
    return sma

In [4]:
with open("./testdata/peaks_tests.pi" ,"rb") as picklein:
    smoothed_diagonal,quick_peaks = pickle.load(picklein)

In [5]:
#needed to load the variables
with open("./testdata/variables.pi","rb") as picklefile:
    frag_index,frag_prop,frag_amount,valid_chroms,chroms_offsets = pickle.load(picklefile)

In [5]:
print(sum([1 if x else 0 for x in quick_peaks]))
print(frag_amount["chr1"])

60176
584662


In [6]:
import re
print(len(re.findall(r'(0+|1+)',"".join(["1" if x else "0" for x in quick_peaks])))/0.16)

63931.25


In [7]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(smoothed_diagonal,range=(0,500), bins=500)
fig.show()

In [6]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(list(itertools.compress(smoothed_diagonal, [not i for i in quick_peaks])),range=(0,500), bins=500)
ax.hist(scipy.stats.poisson.rvs(numpy.mean(smoothed_diagonal), size = 100000),range=(0,500), bins=500)
ax.hist(scipy.stats.poisson.rvs(10, size = 200000),range=(0,500), bins=500)
fig.show()

In [6]:
lengths = [x[3] for x in frag_prop][1:584663]
lengths = moving_integration(lengths, 4)
compress_lengths = list(itertools.compress(lengths, [not i for i in quick_peaks]))
compress_diagonal = list(itertools.compress(smoothed_diagonal, [not i for i in quick_peaks]))
#compress_diagonal = [x for x in compress_diagonal if x>0]

In [26]:

%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.hist(compress_diagonal,range=(0,500), bins=500)
#ax.hist(scipy.stats.binom.rvs(numpy.mean(smoothed_diagonal), size = 100000),range=(0,500), bins=500)
ax.hist(scipy.stats.nbinom.rvs(2.22,0.075, size = 584663),range=(0,500), bins=500)
fig.show()

In [41]:
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()

ax.hist2d(compress_lengths,compress_diagonal,(700,100),range=((0,7000),(0,100)),cmap=matplotlib.pyplot.cm.jet,cmax=90)

fig.show()

ValueError: The dimension of bins must be equal to the dimension of the  sample x.

In [12]:
a = [1,1,1,5,1,1]
b = [|1,1,6,1,1|,1]
moving_integration(a,2)

array([1., 2., 2., 2., 6., 6., 2., 2., 2.])

In [10]:
y = statsmodels.api.nonparametric.lowess(compress_diagonal[:30000], compress_lengths[:30000],return_sorted=False)
%matplotlib qt
fig, ax = matplotlib.pyplot.subplots()
ax.scatter(compress_lengths[:30000],y)
fig.show()

In [1]:
#add some correction for background around like 100kb. 10kb seems hard especially because some fragments are 10 kb

In [21]:
nbinom_data = statsmodels.api.NegativeBinomial(compress_diagonal,numpy.ones(len(compress_diagonal)))

In [22]:
nb = nbinom_data.fit()

Optimization terminated successfully.
         Current function value: 5.019924
         Iterations: 6
         Function evaluations: 14
         Gradient evaluations: 14


In [36]:
blabla =nb.summary()
#const is ln(mean)
#alpha is 1/r
#to get p it's mean = (1-p)*r/p

In [49]:
a = [nbinom_data.pdf(x) for x in range(500)]
matplotlib.pyplot.plot(a)

NotImplementedError: 

In [27]:
numpy.percentile(scipy.stats.nbinom.rvs(2.22,0.075, size = 584663),95)

64.0

In [6]:
numpy.percentile(lengths,99)

NameError: name 'lengths' is not defined

In [40]:
nb.params

array([4.09604928, 1.6148868 ])